In [1]:
import datasets
from datasets import load_dataset
import numpy as np
from scipy.sparse.csgraph import floyd_warshall

In [2]:
en_treebanks = ['en_esl', 'en_ewt', 'en_gum', 'en_gumreddit', 'en_lines', 'en_partut', 'en_pronouns', 'en_pud']

In [3]:
dataset = load_dataset('universal_dependencies', en_treebanks[-1])

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_pud/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
dataset1 = load_dataset('universal_dependencies', 'de_pud')
dataset1

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /users/ujan/.cache/huggingface/datasets/universal_dependencies/de_pud/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1000
    })
})

In [34]:
dataset1['test'][0]

{'idx': 'n01001011',
 'text': '„Ein Großteil des digitalen Übergangs ist für die Vereinigten Staaten neu, ein friedlicher Machtwechsel hingegen nicht“, schrieb Obamas Sonderberaterin Kori Schulman am Montag in einem Blogeintrag.',
 'tokens': ['„',
  'Ein',
  'Großteil',
  'des',
  'digitalen',
  'Übergangs',
  'ist',
  'für',
  'die',
  'Vereinigten',
  'Staaten',
  'neu',
  ',',
  'ein',
  'friedlicher',
  'Machtwechsel',
  'hingegen',
  'nicht',
  '“',
  ',',
  'schrieb',
  'Obamas',
  'Sonderberaterin',
  'Kori',
  'Schulman',
  'am',
  'an',
  'dem',
  'Montag',
  'in',
  'einem',
  'Blogeintrag',
  '.'],
 'lemmas': ['„',
  'ein',
  'Großteil',
  'der',
  'digital',
  'Übergang',
  'sein',
  'für',
  'der',
  'vereinigt',
  'Staat',
  'neu',
  ',',
  'ein',
  'friedlich',
  'Machtwechsel',
  'hingegen',
  'nicht',
  '“',
  ',',
  'schreiben',
  'Obama',
  'Sonderberaterin',
  'Kori',
  'Schulman',
  '_',
  'an',
  'der',
  'Montag',
  'in',
  'ein',
  'Blogeintrag',
  '.'],
 'upos'

In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1000
    })
})

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [35]:
for x in dataset1['test']:
    head = x['head']
    for h in head:
        if h.isdigit():
            if len(head) == int(h):
                print(head)
                

['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['2', '0', '4', '2', '6', '4']


In [40]:
dataset = dataset.filter(lambda example: all(h.isdigit() for h in example['head']))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [41]:
for x in dataset['test']:
    head = x['head']
    for h in head:
        if not h.isdigit():
            #print('not digit')
            #print(h)
            print(head)
            print(len(head))
            break

In [49]:
dataset = dataset.shuffle(seed=42)

In [50]:
def distance_map(batch):
    ids = batch['idx']
    tokens = batch['tokens']
    text = batch['text']
    heads = batch['head']
    
    
    batch_size = len(heads)
    for b in range(batch_size):
        head_list = heads[b]
        print(head_list)
        seq_len = len(head_list)
        print(seq_len)
        graph = np.zeros((seq_len, seq_len))
        # populate adjacency matrix
        for s in range(seq_len):
            if head_list[s] != 0:
                graph[s, int(head_list[s])-1] = 1
        print('graph')
        print(graph)
        dist_matrix = floyd_warshall(csgraph=graph, directed=False, unweighted=True)
        print('dist matrix')
        print(dist_matrix)
        raise

    # compute adj matrix
    
dataset.map(distance_map, batched=True, batch_size=4)

  0%|          | 0/249 [00:00<?, ?ba/s]

['2', '10', '5', '5', '2', '9', '9', '9', '2', '0', '13', '13', '10', '17', '17', '17', '13', '23', '21', '21', '23', '23', '10', '10']
24
graph
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0

RuntimeError: No active exception to reraise

In [48]:
graph

array([[0., 0., 0.],
       [0., 3., 0.],
       [0., 0., 0.]])

In [24]:
a = np.array([
    [0, 1, 0, 0, 0],
    [1, 0, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 0, 0, 1],
    [0, 0, 0, 1, 0]
])

a

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0]])

In [25]:
dist_matrix = floyd_warshall(csgraph=a, directed=False, unweighted=True)
dist_matrix

array([[0., 1., 2., 2., 3.],
       [1., 0., 1., 1., 2.],
       [2., 1., 0., 2., 3.],
       [2., 1., 2., 0., 1.],
       [3., 2., 3., 1., 0.]])

In [21]:
dist_matrix = floyd_warshall(csgraph=graph, directed=False)
dist_matrix

array([[0., 1., 2., 2., 3.],
       [1., 0., 1., 1., 2.],
       [2., 1., 0., 2., 3.],
       [2., 1., 2., 0., 1.],
       [3., 2., 3., 1., 0.]])

In [14]:
graph = np.zeros((3, 3))
graph[1,1] = 3
graph

array([[0., 0., 0.],
       [0., 3., 0.],
       [0., 0., 0.]])

In [57]:
a = np.ones(3)
b = np.zeros(3)
l = []
l.append(a)
l.append(b)
l

[array([1., 1., 1.]), array([0., 0., 0.])]

In [58]:
np.array(l)

array([[1., 1., 1.],
       [0., 0., 0.]])

In [59]:
np.array(l).shape

(2, 3)